In [96]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

import pandas as pd
from pandas.io.json import json_normalize

In [97]:
authenticator = IAMAuthenticator('bkC6hyTVS4qonY34vS0d8Q5F2YY4Kz9m0fyj1cN-5VFw')

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze?version=2019-07-12')

text = """
We bought tickets to both attractions and visited the cloud forest first. It is without a doubt worth every cent - such an incredible experience and you just go at your own pace and take it all in. Stunning displays of beautiful flora enjoyed at many levels. And then there are peaks through to the outside vistas of the Singapore skyline. The flower dome had a beautiful display of dahlias which are one of my favourite flowers and along with some CNY celebratory installations it was enjoyable but doesn’t quite come up to the cloud forest.
Sadly it was raining so we didn’t get to walk around the extended gardens by the bay which could take as long as you want!
Taxi in and out very easy.
"""

response = natural_language_understanding.analyze(
    text=text,
    features=Features(entities=EntitiesOptions(emotion=True, sentiment=True, limit=100000), keywords=KeywordsOptions(emotion=True, sentiment=True, limit=100000))).get_result()

print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 694,
    "features": 2
  },
  "language": "en",
  "keywords": [
    {
      "text": "flower dome",
      "sentiment": {
        "score": 0.929355,
        "label": "positive"
      },
      "relevance": 0.954157,
      "emotion": {
        "sadness": 0.07804,
        "joy": 0.84841,
        "fear": 0.027336,
        "disgust": 0.004205,
        "anger": 0.01156
      },
      "count": 1
    },
    {
      "text": "displays of beautiful flora",
      "sentiment": {
        "score": 0.992443,
        "label": "positive"
      },
      "relevance": 0.898891,
      "emotion": {
        "sadness": 0.029185,
        "joy": 0.890555,
        "fear": 0.006425,
        "disgust": 0.015985,
        "anger": 0.008987
      },
      "count": 1
    },
    {
      "text": "favourite flowers",
      "sentiment": {
        "score": 0.929355,
        "label": "positive"
      },
      "relevance": 0.782343,
      "emotion": {
        "sadness":

In [98]:
keyword_col_names = {
    'text': 'TEXT', 
    'relevance': 'RELEVANCE', 
    'count': 'COUNT', 
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT'
}

keywords_df = json_normalize(response['keywords']).rename(columns=keyword_col_names)
keywords_df.insert(0, 'REVIEW_INDEX', 1234567890)
keywords_df.insert(0, 'WEBSITE_ID', 1)
keywords_df.columns

Index(['WEBSITE_ID', 'REVIEW_INDEX', 'TEXT', 'RELEVANCE', 'COUNT',
       'SENTIMENT_SCORE', 'SENTIMENT_LABEL', 'SADNESS', 'JOY', 'FEAR',
       'DISGUST', 'ANGER', 'MIXED_SENTIMENT'],
      dtype='object')

In [99]:
entities_col_names = {
    'type': 'TYPE',
    'text': 'TEXT', 
    'relevance': 'RELEVANCE', 
    'count': 'COUNT',
    'confidence': 'CONFIDENCE',
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT',
    'disambiguation.name': 'DISAMBIGUATION_NAME',
    'disambiguation.dbpedia_resource': 'DISAMBIGUATION_RESOURCE'
}

entities_df = json_normalize(response['entities']).rename(columns=entities_col_names)
entities_df.insert(0, 'REVIEW_INDEX', 1234567890)
entities_df.insert(0, 'WEBSITE_ID', 1)
entities_df.columns

Index(['WEBSITE_ID', 'REVIEW_INDEX', 'TYPE', 'TEXT', 'RELEVANCE', 'COUNT',
       'CONFIDENCE', 'SENTIMENT_SCORE', 'SENTIMENT_LABEL', 'SADNESS', 'JOY',
       'FEAR', 'DISGUST', 'ANGER', 'DISAMBIGUATION_NAME',
       'DISAMBIGUATION_RESOURCE'],
      dtype='object')